In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import pandas as pd
import re
import random
from tqdm import tqdm

ModuleNotFoundError: No module named 'selenium'

In [ ]:
# initialization
def init_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")
    options.add_argument("--disable-blink-features=AutomationControlled")
    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# scrape JD from linkedin
def scrape_jobs_for_keyword(keyword, num_jobs=5):
    print(f"Scraping JD for keyword: {keyword}")
    driver = init_driver()
    base_url = f"https://www.linkedin.com/jobs/search/?keywords={keyword.replace(' ', '%20')}"
    driver.get(base_url)
    time.sleep(3)

    job_data = []

    while len(job_data) < num_jobs:
        jobs = driver.find_elements(By.CLASS_NAME, 'base-search-card__info')

        for job in jobs:
            try:
                job.click()
                time.sleep(random.uniform(1, 3))
                soup = BeautifulSoup(driver.page_source, 'html.parser')

                title = soup.find("h2", class_="topcard__title").get_text(strip=True)
                company = soup.find("a", class_="topcard__org-name-link").get_text(strip=True)
                description = soup.find("div", class_="show-more-less-html__markup").get_text(strip=True)

                job_data.append({
                    "keyword": keyword,
                    "title": title,
                    "company": company,
                    "description": description
                })

                if len(job_data) >= num_jobs:
                    break
            except:
                continue

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(random.uniform(1.5, 2.5))

    driver.quit()
    return job_data


def normalize_keyword(keyword):
    keyword = keyword.lower()
    keyword = re.sub(r'[\s_/]+', '-', keyword)  
    return keyword


# main function
if __name__ == "__main__":
    keywords = ['INFORMATION-TECHNOLOGY',
                'BUSINESS-DEVELOPMENT',
                'FINANCE',
                'ADVOCATE',
                'ACCOUNTANT']

    all_jobs = []
    for kw in tqdm(keywords, desc="processing"):
        kw = normalize_keyword(kw)
        jobs = scrape_jobs_for_keyword(kw, num_jobs=10)  
        all_jobs.extend(jobs)
        time.sleep(random.uniform(0.5, 1.5))

    df_jobs = pd.DataFrame(all_jobs)
    df_jobs.to_csv("top_JDs.csv", index=False)
    print("Saved JD data to top_JDs.csv")


processing::   0%|          | 0/6 [00:00<?, ?it/s]

Scraping JD for keyword: information-technology
